## Imports

In [6]:
# Imports

import pandas as pd
import numpy as np
import acquire
import prepare

## Call In DF

In [ ]:
urls = acquire.github_geology_urls()
#df = acquire.get_github_geology_results(cached=False)
#df.head
urls

In [4]:
df

""
